# Flash Attention in Torch, Numba and Cuda


We implement in three different ways the Flash Attention forward algorithm from the [Flash Attention 2 paper](https://arxiv.org/pdf/2307.08691). Namely using:

1. Torch
2. Numba
3. Cuda

- We build the kernel for `d=128` and design it so that it computes the full attention with each block running one iteration of the flash attention outer loop.

- We then do some basic performance analysis, showing that on small matrices it performs like torch sdpa and disuss register spiling profiling.

- Finally we run our custom kernel with [Cuda-python](https://developer.nvidia.com/cuda-python) and [Thunder](https://lightning.ai/docs/thunder/latest/).



![./flash_attention_fwd.png](./flash_attention_fwd.png)


## Utils

In [1]:
import numba
from numba.cuda import as_cuda_array as ca
from pathlib import Path
import math
import torch
import sys

sys.path.insert(0, "..")
from utils import load_cuda, get_sig, print_cuda_info

TEST_DIMS = [
    (32, 32),
    (128, 64),
    (512, 512),
    (1024, 512),
]

def get_loaded_cuda_module(fname, verbose=False):
    cuda_src_path = f"./{fname}.cu"
    torch_src_path = f"./torch_extension_template.cu"
    cuda_src = Path(cuda_src_path).read_text()
    cuda_src += Path(torch_src_path).read_text()
    cuda_src = cuda_src.replace("your_function_name", fname)
    cpp_src = get_sig(fname, cuda_src)
    return load_cuda(cuda_src, cpp_src, [fname], verbose=verbose)


def check_close(O, O_expected, L=None, L_expected=None, atol=5*1e-5):
    O_diff = (O - O_expected).abs().max()
    print("Max absolute difference:")
    if atol:
        assert O_diff < atol, f"O diff too large: {O_diff} > {atol=}"
    print("O: ", O_diff)
    if L is not None:
        L_diff = (L.squeeze() - L_expected).abs().max()
        if atol:
            assert L_diff < atol, f"L diff too large: {L_diff} > {atol=}"
        print("L: ", L_diff)

# Test tensors
def get_test_tensors(N_inp, N_out, d):
    Q = torch.randn(N_out, d).contiguous().to("cuda")
    K = torch.randn(N_inp, d).contiguous().to("cuda")
    V = torch.randn(N_inp, d).contiguous().to("cuda")
    scaling = 1.0 / math.sqrt(d)

    # Get expected O
    O_expected = torch.nn.functional.scaled_dot_product_attention(Q, K, V)
    S = (Q @ K.T) * scaling  # shape: (N_out, N_inp)
    L_expected = torch.logsumexp(S, dim=-1)
    return Q, K, V, scaling, O_expected, L_expected

# Check test tensors
N_inp = 512
N_out = 512
d = 128

Q, K, V, scaling, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)
check_close(O=torch.softmax(Q @ K.T * scaling, dim=-1) @ V, O_expected=O_expected)

Max absolute difference:
O:  tensor(1.1921e-07, device='cuda:0')


## Torch

In [2]:
def flash_attention_torch(Q, K, V, O, L, N_inp, N_out, d) -> None:
    B_c = 16
    B_r = 16
    T_c = (N_inp + B_c - 1) // B_c
    T_r = (N_out + B_r - 1) // B_r

    scaling = 1 / math.sqrt(d)

    # Q and O L split into T_r; K, V in T_c blocks
    for i in range(T_r):
        Q_i = Q[i * B_r : (i + 1) * B_r]
        O_i = torch.zeros(B_r, d)
        L_i = torch.zeros(B_r, 1)
        m_i = torch.full((B_r, 1), -math.inf)
        last_m_i = m_i
        for j in range(T_c):
            K_j = K[j * B_c : (j + 1) * B_c]
            V_j = V[j * B_c : (j + 1) * B_c]
            S_i = scaling * (Q_i @ K_j.T)
            m_i = torch.maximum(m_i, S_i.max(dim=-1, keepdim=True).values)
            P_i = torch.exp(S_i - m_i)
            L_i = torch.exp(last_m_i - m_i) * L_i + P_i.sum(dim=-1, keepdim=True)
            O_i = torch.exp(last_m_i - m_i) * O_i + P_i @ V_j
            last_m_i = m_i
        O_i = (1.0 / L_i) * O_i
        L_i = m_i + torch.log(L_i)
        O[i * B_r : (i + 1) * B_r] = O_i
        L[i * B_r : (i + 1) * B_r] = L_i

In [3]:
O_torch_loop = torch.zeros(N_out, d)
L_torch_loop = torch.zeros(N_out, 1)

flash_attention_torch(Q.to("cpu"), K.to("cpu"), V.to("cpu"), O_torch_loop, L_torch_loop, N_inp, N_out, d)

check_close(
    O_torch_loop.to("cuda"), 
    O_expected,
    L_torch_loop.to("cuda"),
    L_expected
)

Max absolute difference:
O:  tensor(3.2037e-07, device='cuda:0')
L:  tensor(9.5367e-07, device='cuda:0')


## Numba

Tiling strategy: threads cooperate loading shared arrays in each `i, j` loop and write their own entries of `l_i, m_i, O_i`

### All arrays in shared memory

In [4]:
@numba.cuda.jit
def flash_attention_numba_all_smem(Q, K, V, scaling: numba.float32, L, O, N_out, N_inp):
    B_c = 16
    B_r = 16
    T_c = (N_inp + B_c - 1) // B_c
    T_r = (N_out + B_r - 1) // B_r
    inp_dtype = K.dtype
    tid_x = numba.cuda.threadIdx.x
    tid_y = numba.cuda.threadIdx.y

    Q_i = numba.cuda.shared.array((B_r, d), inp_dtype)
    K_j = numba.cuda.shared.array((B_c, d), inp_dtype)
    V_j = numba.cuda.shared.array((B_c, d), inp_dtype)
    S = numba.cuda.shared.array((B_r, B_c), inp_dtype)
    # These can be in registers but wont fit too large
    l_i = numba.cuda.shared.array((B_r,), inp_dtype)
    m_i = numba.cuda.shared.array((B_r,), inp_dtype)
    O_i = numba.cuda.shared.array((B_r, d), inp_dtype)

                 
    for i in range(T_r):
        for ii in range(tid_y, B_r, numba.cuda.blockDim.y):
            for dd in range(tid_x, d, numba.cuda.blockDim.x):
                Q_i[ii, dd] = Q[ii + i * B_r, dd]
                O_i[ii, dd] = 0
            l_i[ii] = 0
            m_i[ii] = -math.inf
        numba.cuda.syncthreads()

        for j in range(T_c):
            for jj in range(tid_y, B_c, numba.cuda.blockDim.y):
                for dd in range(tid_x, d, numba.cuda.blockDim.x):
                    K_j[jj, dd] = K[jj + j * B_c, dd]
                    V_j[jj, dd] = V[jj + j * B_c, dd]

            # S[ii][jj] = scaling * (Q_i @ K_j.T)
            numba.cuda.syncthreads()
            for ii in range(tid_x, B_r, numba.cuda.blockDim.x):
                for jj in range(tid_y, B_c, numba.cuda.blockDim.y):
                    S_ij = 0
                    for dd in range(d):
                        S_ij += Q_i[ii, dd] * K_j[jj, dd]
                    S_ij = scaling * S_ij
                    S[ii][jj] = S_ij

            numba.cuda.syncthreads()
            for ii in range(tid_y, B_r, numba.cuda.blockDim.y):
                m = m_i[ii]
                last_m = m
                for jj in range(B_c):
                    m = max(m, S[ii][jj])
                m_i[ii] = m
                l = math.exp(last_m - m) * l_i[ii]

                for dd in range(tid_x, d, numba.cuda.blockDim.x):
                    O_i[ii, dd] *= math.exp(last_m - m)
                for jj in range(B_c):
                    P_ij = math.exp(S[ii][jj] - m)
                    l += P_ij
                    for dd in range(tid_x, d, numba.cuda.blockDim.x):
                        O_i[ii, dd] += P_ij * V_j[jj, dd]
                l_i[ii] = l
                
        for ii in range(tid_y, B_r, numba.cuda.blockDim.y):  
            for dd in range(tid_x, d, numba.cuda.blockDim.x):
                O[ii + i * B_r, dd] = O_i[ii, dd] / l_i[ii]
            L[ii + i * B_r] = m_i[ii] + math.log(l_i[ii])
        
        numba.cuda.syncthreads()

In [5]:

block_dim_x = 32
block_dim_y = 16

for N_inp, N_out in TEST_DIMS:

    Q, K, V, scaling, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)
    O_all_smem = torch.zeros(N_out, d, device="cuda").contiguous()
    L_all_smem = torch.zeros(N_out, device="cuda")
    tpb = (block_dim_x, block_dim_y)
    grid = (1,)
    flash_attention_numba_all_smem[grid, tpb](Q, K, V, scaling, L_all_smem, O_all_smem,  N_out, N_inp)
    torch.cuda.synchronize()
    check_close(
        O_all_smem, 
        O_expected,
        L_all_smem,
        L_expected,
    )

/opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Max absolute difference:
O:  tensor(7.1526e-07, device='cuda:0')
L:  tensor(4.7684e-07, device='cuda:0')
Max absolute difference:
O:  tensor(3.5763e-07, device='cuda:0')
L:  tensor(9.5367e-07, device='cuda:0')
Max absolute difference:
O:  tensor(4.1723e-07, device='cuda:0')
L:  tensor(9.5367e-07, device='cuda:0')
Max absolute difference:
O:  tensor(4.9174e-07, device='cuda:0')
L:  tensor(1.9073e-06, device='cuda:0')


### Moving `m_i`, `l_i`, `O_i` to registers

Current shared-memory usage across threads:
```
Shar = (B_r * d * 4) # Q_i
+ (B_c * d * 4) # K_j
+ (B_c * d * 4) # V_j
+ (B_r * B_c * 4) # S
= ~25 KB
```

Current block-shared accumulators (`m_i`, `l_i`, `O_i`):
```
Loc = 4 * (B_r + B_r + (B_r * d)) = 8320 B ≈ 8 KB
```

Total shared usage: **~33 KB** (fine for 1 block/SM).

---

**Idea:** Move `m_i`, `l_i`, `O_i` to *per-thread* locals to fit in registers.

Problem: Full-size per-thread arrays would need

```
Loc * 32 * 16 ≈ 4 MB > 256 KB register memory per SM
```

---

**Optimization:** With tiling `tpb = (32, 16)`:

- Each thread handles only  
  `d // blockDim.x = 4` columns in `x`  
  `B_r // blockDim.y = 1` row in `y`
- So per-thread locals can be much smaller:

```python
l_i = numba.cuda.local.array((1,), inp_dtype)   # 4 B
m_i = numba.cuda.local.array((1,), inp_dtype)   # 4 B
O_i = numba.cuda.local.array((4,), inp_dtype)   # 16 B
```
-> Per-thread = 24 B, per block = 24 * 32 * 16 = 12 KB < 256 KB -> avoid register pressure and spills.


This is how we set up `flash_attention_numba` below and the cuda version in [`./flash_attention.cu`](./flash_attention.cu)

In the performance section we run `./flash_attention_spilling_from_registers.cu` that fits full arrays in local variables, to show the performance decrease by slowing the kernel by ~3×

In [6]:
# Note that we also parallelised the loops in `T_r`, 
# so that each block processes only one outer loop
# and we run a grid of T_r size

block_dim_x = 64
block_dim_y = 8
B_r = 8
B_c = 64
d_over_dim_x = d // block_dim_x
B_r_over_dim_y = B_r // block_dim_y

@numba.cuda.jit
def flash_attention_numba(Q, K, V, scaling: numba.float32, L, O, N_out, N_inp):
    B_c = 16
    B_r = 8
    T_c = (N_inp + B_c - 1) // B_c
    # T_r = (N_out + B_r - 1) // B_r
    inp_dtype = K.dtype
    tid_x = numba.cuda.threadIdx.x
    tid_y = numba.cuda.threadIdx.y
    dim_y = numba.cuda.blockDim.y
    dim_x = numba.cuda.blockDim.x
    block_id_x = numba.cuda.blockIdx.x
    

    Q_i = numba.cuda.shared.array((B_r, d), inp_dtype)
    K_j = numba.cuda.shared.array((B_c, d), inp_dtype)
    V_j = numba.cuda.shared.array((B_c, d), inp_dtype)
    S = numba.cuda.shared.array((B_r, B_c), inp_dtype)

    l_i = numba.cuda.local.array((B_r_over_dim_y,), inp_dtype)
    m_i = numba.cuda.local.array((B_r_over_dim_y,), inp_dtype)
    O_i = numba.cuda.local.array((B_r_over_dim_y, d_over_dim_x), inp_dtype)

                 
    for i in range(block_id_x, block_id_x+1):
        for ii in range(tid_y, B_r, dim_y):
            for dd in range(tid_x, d, dim_x):
                Q_i[ii, dd] = Q[ii + i * B_r, dd]

        for ii in range(B_r_over_dim_y):
            for dd in range(d_over_dim_x):
                O_i[ii, dd] = 0
            l_i[ii] = 0
            m_i[ii] = -math.inf
        
        numba.cuda.syncthreads()
        for j in range(T_c):
            for jj in range(tid_y, B_c, dim_y):
                for dd in range(tid_x, d, dim_x):
                    K_j[jj, dd] = K[jj + j * B_c, dd]
                    V_j[jj, dd] = V[jj + j * B_c, dd]

            # S[ii][jj] = scaling * (Q_i @ K_j.T)
            numba.cuda.syncthreads()
            for ii in range(tid_x, B_r, dim_x):
                for jj in range(tid_y, B_c, dim_y):
                    S_ij = 0
                    for dd in range(d):
                        S_ij += Q_i[ii, dd] * K_j[jj, dd]
                    S_ij = scaling * S_ij
                    S[ii][jj] = S_ij

            numba.cuda.syncthreads()
            for ii in range(B_r_over_dim_y):
                m = m_i[ii]
                last_m = m
                for jj in range(B_c):
                    m = max(m, S[ii * dim_y + tid_y][jj])
                m_i[ii] = m
                l = numba.float32(math.exp(last_m - m)) * l_i[ii]

                for dd in range(d_over_dim_x):
                    O_i[ii, dd] *= numba.float32(math.exp(last_m - m))
                for jj in range(B_c):
                    P_ij = numba.float32(math.exp(S[ii * dim_y + tid_y][jj] - m))
                    l += P_ij
                    for dd in range(d_over_dim_x):
                        O_i[ii, dd] += P_ij * V_j[jj, dd * dim_x + tid_x]
                l_i[ii] = l
                
        for ii in range(B_r_over_dim_y):  
            for dd in range(d_over_dim_x):
                O[ii * dim_y + tid_y + i * B_r, dd * dim_x + tid_x] = O_i[ii, dd] / l_i[ii]
            L[ii * dim_y + tid_y + i * B_r] = m_i[ii] + numba.float32(math.log(l_i[ii]))

In [7]:
for N_inp, N_out in TEST_DIMS:
    Q, K, V, scaling, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)

    O_all_smem = torch.zeros(N_out, d, device="cuda").contiguous()
    L_all_smem = torch.zeros(N_out, device="cuda")
    tpb = (block_dim_x, block_dim_y)
    T_r = (N_out + B_r - 1) // B_r
    grid = (T_r,)
    flash_attention_numba[grid, tpb](Q, K, V, scaling, L_all_smem, O_all_smem,  N_out, N_inp)
    check_close(
        O_all_smem, 
        O_expected,
        L_all_smem,
        L_expected,
    )

/opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Max absolute difference:
O:  tensor(3.5763e-07, device='cuda:0')
L:  tensor(4.7684e-07, device='cuda:0')
Max absolute difference:
O:  tensor(3.2783e-07, device='cuda:0')
L:  tensor(9.5367e-07, device='cuda:0')
Max absolute difference:
O:  tensor(4.0233e-07, device='cuda:0')
L:  tensor(1.4305e-06, device='cuda:0')
Max absolute difference:
O:  tensor(3.9488e-07, device='cuda:0')
L:  tensor(1.9073e-06, device='cuda:0')


/opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/opt/conda/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


## Cuda

### flash_attention_numba in Cuda

See [flash_attention.cu](./flash_attention.cu)

In [8]:
fname = "flash_attention"
module = get_loaded_cuda_module(fname)

/opt/conda/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2073: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [9]:
for N_inp, N_out in TEST_DIMS:
    Q, K, V, _, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)
    O_move_registers, L_move_registers = getattr(module, fname)(Q, K, V)
    check_close(
        O_move_registers, 
        O_expected,
        L_move_registers,
        L_expected,
    )

Max absolute difference:
O:  tensor(4.4703e-07, device='cuda:0')
L:  tensor(4.7684e-07, device='cuda:0')
Max absolute difference:
O:  tensor(4.1723e-07, device='cuda:0')
L:  tensor(4.7684e-07, device='cuda:0')
Max absolute difference:
O:  tensor(5.6624e-07, device='cuda:0')
L:  tensor(9.5367e-07, device='cuda:0')
Max absolute difference:
O:  tensor(3.7253e-07, device='cuda:0')
L:  tensor(1.9073e-06, device='cuda:0')


## Triton

In [10]:
# import torch
# import triton
# import triton.language as tl
# import math


# @triton.jit
# def flash_attn_triton(
#     Q, K, V, O, L,
#     M: tl.constexpr, N: tl.constexpr, D: tl.constexpr,
#     stride_qm, stride_qd,
#     stride_km, stride_kd,
#     stride_vm, stride_vd,
#     stride_om, stride_od,
#     scaling: tl.constexpr,
#     BLOCK_M: tl.constexpr,      # >=16
#     BLOCK_N: tl.constexpr,      # >=16
#     BLOCK_D_TILE: tl.constexpr, # >=16 (for tl.dot inner dim)
# ):
#     pid_m = tl.program_id(0)
#     offs_m = pid_m * BLOCK_M + tl.arange(0, BLOCK_M)

#     # per-program accumulators
#     m_i = tl.full((BLOCK_M,), -float("inf"), dtype=tl.float32)
#     l_i = tl.zeros((BLOCK_M,), dtype=tl.float32)
#     acc = tl.zeros((BLOCK_M, D), dtype=tl.float32)  # D small in your use-case (128)

#     # Precompute Q chunks for reuse across K tiles:
#     # We'll load Q in D-chunks on demand inside the D loop below.

#     # Sweep over K/V tiles (columns)
#     for start_n in range(0, N, BLOCK_N):
#         offs_n = start_n + tl.arange(0, BLOCK_N)
#         # Build scores for this (BLOCK_M x BLOCK_N) tile by accumulating across D chunks
#         scores_tile = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)

#         # accumulate scores across D in tiles so each tl.dot sees K >= 16
#         for start_d in range(0, D, BLOCK_D_TILE):
#             offs_d = start_d + tl.arange(0, BLOCK_D_TILE)

#             # load Q chunk: [BLOCK_M, D_chunk]
#             q_ptrs = Q + (offs_m[:, None] * stride_qm + offs_d[None, :] * stride_qd)
#             q_mask = (offs_m[:, None] < M) & (offs_d[None, :] < D)
#             q_chunk = tl.load(q_ptrs, mask=q_mask, other=0.0).to(tl.float32)

#             # load K chunk: [BLOCK_N, D_chunk]
#             k_ptrs = K + (offs_n[:, None] * stride_km + offs_d[None, :] * stride_kd)
#             k_mask = (offs_n[:, None] < N) & (offs_d[None, :] < D)
#             k_chunk = tl.load(k_ptrs, mask=k_mask, other=0.0).to(tl.float32)

#             # partial scores for this d-chunk: [BLOCK_M, BLOCK_N]
#             # tl.dot(q_chunk, k_chunk.T) requires D_chunk >= 16 for tensorcore lowering
#             partial = tl.dot(q_chunk, tl.trans(k_chunk))
#             scores_tile = scores_tile + partial * scaling

#         # mask invalid columns (columns beyond N)
#         col_idxs = start_n + tl.arange(0, BLOCK_N)
#         col_mask = col_idxs[None, :] < N
#         # masked max (invalid columns must not influence max)
#         scores_masked = tl.where(col_mask, scores_tile, tl.full(scores_tile.shape, -float("inf"), dtype=tl.float32))
#         # numerically stable online-softmax update for this tile
#         m_ij = tl.max(scores_masked, axis=1)            # per-row max for this tile
#         m_new = tl.maximum(m_i, m_ij)
#         # exponent factor for old -> new max
#         alpha = tl.exp(m_i - m_new)                      # shape [BLOCK_M]
#         # compute probabilities p = exp(scores_tile - m_new[:,None]) with mask
#         p = tl.where(col_mask, tl.exp(scores_tile - m_new[:, None]), tl.zeros_like(scores_tile))

#         # update l_i (scalar per row) and acc (vector per row over D)
#         # We need contribution = p @ V_tile (shape [BLOCK_M, D]), we compute it by D-chunking V.
#         contrib = tl.zeros((BLOCK_M, D), dtype=tl.float32)

#         for start_d in range(0, D, BLOCK_D_TILE):
#             offs_d = start_d + tl.arange(0, BLOCK_D_TILE)

#             # load V chunk: [BLOCK_N, D_chunk]
#             v_ptrs = V + (offs_n[:, None] * stride_vm + offs_d[None, :] * stride_vd)
#             v_mask = (offs_n[:, None] < N) & (offs_d[None, :] < D)
#             v_chunk = tl.load(v_ptrs, mask=v_mask, other=0.0).to(tl.float32)

#             # p @ v_chunk -> [BLOCK_M, D_chunk]
#             contrib_chunk = tl.dot(p, v_chunk)

#             # write contrib_chunk into the right slice of contrib (per-D)
#             # We can't slice assign in Triton, so build a zero buffer and add shifted
#             # we'll accumulate into contrib by adding at correct columns via masking and pointer maths
#             # Prepare pointers into a fake global buffer: use O as temporary workspace ptrs (safe as we don't rely on O until the end)
#             # But simpler: accumulate into contrib full matrix and then later use it.
#             # Build a tile-sized mask and place chunk at [ :, start_d:start_d+D_chunk ]
#             # We'll build a small tensor temp_full of shape (BLOCK_M, BLOCK_D_TILE) and then write into contrib via elementwise add.
#             # Instead of complex pointer arithmetic, we do a python-level placement emulation:
#             # create a view on contrib for the slice and add (Triton currently allows direct arithmetic with slices using offsets)
#             # compute column global offsets for these chunk columns
#             col_offset = start_d
#             # We'll do elementwise add by computing an index matrix and mask store:
#             offs_d_global = col_offset + tl.arange(0, BLOCK_D_TILE)   # shape [BLOCK_D_TILE]
#             mask_d = offs_d_global[None, :] < D
#             # compute pointers into a pseudo buffer using O as base (but we want to avoid clobbering O mid-kernel)
#             # Simpler: directly accumulate into contrib by constructing a broadcast and using `tl.store` into a local slice.
#             # Triton does not support direct writes into a local Python array slice; instead accumulate by adding into contrib using broadcasting
#             # We'll rebuild contrib with addition over the slice:
#             # Build zeros buffer of same shape as contrib and add contribution in place by combining masks:
#             # Efficient and simpler: use `tl.where` to add contrib_chunk into the appropriate columns
#             # Build an index matrix for columns: col_idx_mat [1, D] where positions in this chunk are True
#             col_idx_mat = offs_d_global[None, :]  # shape [1, BLOCK_D_TILE]
#             # Create boolean mask of size (1, D) for these columns; then broadcast to (BLOCK_M, D)
#             # But Triton doesn't allow creating a mask of length D easily. Simpler: implement contrib as accumulation per chunk into acc directly:
#             # acc[:, start_d:start_d+D_chunk] += contrib_chunk
#             # We can store the chunk to global memory O temporarily and then later read; that's messy.
#             # Given D is small (128) we can instead hold contrib as a flat contiguous buffer and write chunk via pointer math:
#             # We'll compute pointer into a per-program scratch area in global memory represented by O + big offset.
#             # To avoid complexity, we will construct a temporary full-width tile with zeros and place contrib_chunk into columns [start_d:start_d+D_chunk] using masks:
#             # Create temp_tile = zeros((BLOCK_M, D))
#             temp_tile = tl.zeros((BLOCK_M, D), dtype=tl.float32)
#             # Build pointers for temp tile positions (this is only conceptual; we cannot create a huge temp easily)
#             # Instead we use an element-wise construction: for each column in BLOCK_D_TILE, place contrib_chunk column into the corresponding global column position
#             for k_col in range(0, BLOCK_D_TILE):
#                 col = start_d + k_col
#                 valid_col = col < D
#                 if valid_col:
#                     # add contrib_chunk[:, k_col] into contrib[:, col]
#                     # Build a small vector and perform elementwise addition through reassigning contrib via tl.where trick:
#                     # get col values of contrib (current)
#                     # we can't index contrib[:, col] as an lvalue; instead we do:
#                     # contrib = contrib + (mask_column * expand(contrib_chunk[:, k_col]))
#                     # mask_column shape: (1,D) with True at position col. We'll build mask by comparing an arange to col.
#                     cols_all = tl.arange(0, D)
#                     mask_col_all = (cols_all[None, :] == col)
#                     # broadcast contrib_chunk column
#                     to_add = contrib_chunk[:, k_col][:, None] * mask_col_all
#                     contrib = contrib + to_add
#                 # else ignore

#         # After contrib computed (shape [BLOCK_M, D]) and p ready
#         # Update acc and l_i
#         l_new = alpha * l_i + tl.sum(p, axis=1)
#         acc = (alpha[:, None] * acc) + contrib
#         m_i = m_new
#         l_i = l_new

#     # Write outputs O (normalize) and L
#     out = acc / l_i[:, None]
#     # store O
#     offs_d_full = tl.arange(0, D)
#     o_ptrs = O + (offs_m[:, None] * stride_om + offs_d_full[None, :] * stride_od)
#     mask_o = (offs_m[:, None] < M) & (offs_d_full[None, :] < D)
#     tl.store(o_ptrs, out, mask=mask_o)
#     # store L
#     tl.store(L + offs_m, m_i + tl.log(l_i), mask=offs_m < M)


# def flash_attention_triton(Q, K, V):
#     """
#     Q, K, V: torch tensors [N_out, d], [N_inp, d], [N_inp, d]
#              (fp16/bf16/fp32). Returns O [N_out, d], L [N_out].
#     scaling: float (e.g., 1/sqrt(d))
#     """
#     assert Q.is_cuda and K.is_cuda and V.is_cuda
#     assert Q.shape[1] == K.shape[1] == V.shape[1]
#     M, D = Q.shape
#     N = K.shape[0]
#     scaling = 1./math.sqrt(Q.shape[1])

#     # output buffers
#     O = torch.empty_like(Q, dtype=torch.float32)  # accum in fp32; cast later if you want
#     L = torch.empty(M, device=Q.device, dtype=torch.float32)

#     # tile sizes (match your numba defaults if you like)
#     BLOCK_M = 16    # B_r
#     BLOCK_N = 16   # B_c
#     BLOCK_D = D    # set to head dim; Triton specializes per call

#     # launch grid: one program per BLOCK_M rows
#     grid = (triton.cdiv(M, BLOCK_M),)

#     flash_attn_triton[grid](
#         Q, K, V, O, L,
#         M, N, D,
#         Q.stride(0), Q.stride(1),
#         K.stride(0), K.stride(1),
#         V.stride(0), V.stride(1),
#         O.stride(0), O.stride(1),
#         float(scaling),
#         BLOCK_M=BLOCK_M, BLOCK_N=BLOCK_N, BLOCK_D_TILE=BLOCK_D,
#         num_warps=4,  # good starting point; tune as needed
#         num_stages=2,
#     )

#     return O, L


In [11]:
# for N_inp, N_out in TEST_DIMS:
#     Q, K, V, scaling, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)

#     O_triton = torch.zeros(N_out, d, device="cuda").contiguous()
#     L_triton = torch.zeros(N_out, device="cuda")
  
#     flash_attention_triton(Q, K, V)
#     check_close(
#         O_triton, 
#         O_expected,
#         L_triton,
#         L_expected,
#     )

## Performance 

Run timeit on different dimensions. 

- Recall that we build the kernel for `d=128` and design it so that it computes the full attention in a single block.

- For matrices with small `N_out` this implementation is comparable with `scaled_dot_product_attention` faster backends.

- But by making N larger this implementation slows down dramatically it only uses a single block.

- Note that the register splilling version is always much slower than `scaled_dot_product_attention`.


### Load registers spilling version

This is the version loading full arrays as local variables in threads, which leads to spilling.

In [12]:
fname_spill_from_registers = "flash_attention_spilling_from_registers"
module_spilling_from_registers = get_loaded_cuda_module(fname_spill_from_registers)
O_cuda_spilling, L_cuda_spilling = getattr(module_spilling_from_registers, fname_spill_from_registers)(Q, K, V)
check_close(O_cuda_spilling, O_expected, L_cuda_spilling, L_expected)

/opt/conda/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2073: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Max absolute difference:
O:  tensor(3.7253e-07, device='cuda:0')
L:  tensor(1.9073e-06, device='cuda:0')


### Timeit: sdpa backends

In [13]:
# Benchmark first the different backends for scaled_dot_product_attention
# For small tensors
from torch.nn.attention import sdpa_kernel, SDPBackend

Q, K, V, _, _, _ = get_test_tensors(N_inp=2048, N_out=2048, d=128)
Qhalf, Khalf, Vhalf = Q.to(torch.float16), K.to(torch.float16), V.to(torch.float16)

def run_sdpa_with_backend(Q, K, V, backend):
    with sdpa_kernel(backends=backend):
        torch.nn.functional.scaled_dot_product_attention(
            Q.unsqueeze(0).unsqueeze(0), 
            K.unsqueeze(0).unsqueeze(0), 
            V.unsqueeze(0).unsqueeze(0),
        )
    torch.cuda.synchronize()

print("\nExplicit softmax(Q @ K.T * scaling) @ V")
%timeit torch.softmax(Q @ K.T * scaling, dim=-1) @ V; torch.cuda.synchronize()
print("\nBackend: None")
%timeit torch.nn.functional.scaled_dot_product_attention(Q, K, V)
print("\nBackend: EFFICIENT_ATTENTION")
%timeit run_sdpa_with_backend(Q, K, V, backend=SDPBackend.EFFICIENT_ATTENTION)
print("\nBackend: FLASH_ATTENTION (f16)")
%timeit run_sdpa_with_backend(Qhalf, Khalf, Vhalf, backend=SDPBackend.FLASH_ATTENTION)
print("\nBackend: CUDNN_ATTENTION (f16)")
%timeit run_sdpa_with_backend(Qhalf, Khalf, Vhalf, backend=SDPBackend.CUDNN_ATTENTION)


Explicit softmax(Q @ K.T * scaling) @ V


334 μs ± 521 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Backend: None
379 μs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Backend: EFFICIENT_ATTENTION
589 μs ± 873 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Backend: FLASH_ATTENTION (f16)
111 μs ± 322 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Backend: CUDNN_ATTENTION (f16)
162 μs ± 27.8 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Timeit: sdpa vs custom kernel

In [14]:
# Check now run against 
for N_inp, N_out in TEST_DIMS:
    Q, K, V, _, _, _ = get_test_tensors(N_inp, N_out, d)
    print(f"\n\n**********\nDimensions: {N_out=}, {N_inp=}, {d=}")
    torch.cuda.synchronize()
    print("\n- Torch scaled_dot_product_attention")
    %timeit torch.nn.functional.scaled_dot_product_attention(Q, K, V); torch.cuda.synchronize()
    print("\n- Custom Flash Attention")
    %timeit getattr(module, fname)(Q, K, V); torch.cuda.synchronize()
    print("\n- Custom Flash Attention: spill from registers")
    %timeit getattr(module_spilling_from_registers, fname_spill_from_registers)(Q, K, V); torch.cuda.synchronize()



**********
Dimensions: N_out=32, N_inp=32, d=128

- Torch scaled_dot_product_attention
104 μs ± 868 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

- Custom Flash Attention
39.3 μs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

- Custom Flash Attention: spill from registers
313 μs ± 659 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


**********
Dimensions: N_out=64, N_inp=128, d=128

- Torch scaled_dot_product_attention
106 μs ± 826 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

- Custom Flash Attention
76.5 μs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

- Custom Flash Attention: spill from registers
2.26 ms ± 1.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**********
Dimensions: N_out=512, N_inp=512, d=128

- Torch scaled_dot_product_attention
112 μs ± 731 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

- Custom Flash Attention
229 μs ± 735 ns per loop (mean ± std. dev. of 

### Profile

We can compare the performance of the kernel with and without register spilling as follows:
```
# Get ptx files or use https://godbolt.org/
nvcc -ptx flash_attention.cu -o flash_attention.ptx
nvcc -ptx flash_attention_spilling_from_registers.cu -o flash_attention_spilling_from_registers.ptx

# Get ncu metrics
nvcc -O3 -o test_attention main.cu flash_attention.cu flash_attention_spilling_from_registers.cu
ncu ./test_attention
```

#### PTX comparison

For the [spilling kernel](./flash_attention_spilling_from_registers.cu) we see 
```ptx
.local .align 16 .b8 	__local_depot0[8320];
```
where 8320 B is exactly the allocation for
```cu
float l_i[B_r];
float m_i[B_r];
float O_i[B_r][d];
```


#### Nsight Compute Comparison

| **Metric** | **Spilling Kernel (`flash_attention_spilling_from_registers_k`)** | **Non-Spilling Kernel (`flash_attention_k`)** | **Difference** |
|------------|------------------------------------------------------------------|-----------------------------------------------|---------------------------------|
| **Duration** | **13.02 ms** | **2.10 ms** | Spilling kernel is ~6× slower |
| **Compute (SM) Throughput** | **0.26%** | **1.20%** | 5× higher compute utilization in non-spilling kernel |
| **L2 Cache Throughput** | **1.71%** | **0.56%** | Spilling kernel hits L2 more |

## Cuda-Python

`conda install conda-forge::cuda-python`

In [18]:
# Create program
from cuda import cuda, nvrtc

cuda_src_path = f"./flash_attention.cu"
cuda_src = Path(cuda_src_path).read_text()

N_inp = 32
N_out = 32
d = 128
B_r, B_c = 16, 16
T_r = (N_out + B_r -1) // B_r
T_c = (N_inp + B_r -1) // B_c
Q, K, V, scale_factor, O_expected, L_expected = get_test_tensors(N_inp, N_out, d)

err, prog = nvrtc.nvrtcCreateProgram(str.encode(cuda_src), b"flash_attention.cu", 0, [], [])

# Compile program
min, maj = torch.cuda.get_device_capability()
opts = [
    f"--gpu-architecture=compute_{min}{maj}".encode(), 
    "--device-as-default-execution-space".encode(),
    "--std=c++14".encode()]
err, = nvrtc.nvrtcCompileProgram(prog, len(opts), opts)

print(err)

# Get PTX from compilation
err, ptxSize = nvrtc.nvrtcGetPTXSize(prog)
ptx = b" " * ptxSize
err, = nvrtc.nvrtcGetPTX(prog, ptx)
print(err)

err, logSize = nvrtc.nvrtcGetProgramLogSize(prog)
log = b" " * logSize
err, = nvrtc.nvrtcGetProgramLog(prog, log)
print(log.decode())
# print(ptx.decode())

# Load PTX as module data and retrieve function
err, module = cuda.cuModuleLoadData(ptx)
print(err)
err, kernel = cuda.cuModuleGetFunction(module, b"flash_attention_k")
print(err, kernel)

# Allocate tensors
# S3 = torch.zeros(N_out, N_out, device="cuda")
O_cuda_py = torch.zeros(N_out, d, device="cuda")
L_cuda_py = torch.zeros(N_out, device="cuda")

# To quote the official tutorial: (https://nvidia.github.io/cuda-python/overview.html)
# The following code example is not intuitive
# Subject to change in a future release

int_args = torch.tensor([0, T_r, T_c], dtype=torch.int32)
float_args = torch.tensor([scale_factor], dtype=torch.float32)
ptr_args = torch.tensor([i.data_ptr() for i in (O_cuda_py, L_cuda_py, Q, K, V)], dtype=torch.uint64)

args = torch.tensor([
    *(i.data_ptr() for i in ptr_args),
    *(i.data_ptr() for i in float_args),
    *(i.data_ptr() for i in int_args)], dtype=torch.uint64)

args

0
0
 
0
0 <CUfunction 0x55669f57f960>


<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.
<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


tensor([93899257721792, 93899257721800, 93899257721808, 93899257721816,
        93899257721824, 93899250177280, 93899249702528, 93899249702532,
        93899249702536], dtype=torch.uint64)

In [19]:
def fn():
    err = cuda.cuLaunchKernel(
        kernel,
        T_r,  # grid x dim
        1,  # grid y dim
        1,  # grid z dim
        16,  # block x dim
        16,  # block y dim
        1,  # block z dim
        0,  # dynamic shared memory
        torch.cuda.current_stream().stream_id,  # stream
        args.data_ptr(),  # kernel arguments
        0,  # extra (ignore)
    )

fn()

(O_cuda_py - O_expected).abs().max()

tensor(1.2352, device='cuda:0')

In [20]:
print(f"\n\n**********\nDimensions: {N_out=}, {N_inp=}, {d=}")
torch.cuda.synchronize()
print("\n- Custom Flash Attention: Cuda-python")
%timeit fn(); torch.cuda.synchronize()



**********
Dimensions: N_out=32, N_inp=32, d=128

- Custom Flash Attention: Cuda-python
35.3 μs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Thunder

[Installation guide](https://lightning.ai/docs/thunder/latest/fundamentals/installation.html)

We use thunder here to include our custom kernel as an option within torch's own `scaled_dot_product_attention`.

In [21]:
import thunder

attn_ex = thunder.extend.OperatorExecutor('attn_ex', version=0.01)
thunder.add_default_executor(attn_ex)

# [attn_ex, attn_ex, sdpa, nvfuser]

def my_attn_impl(query, key, value, scale):
    n_out, d = query.shape

    # S3 = torch.zeros(N_out, N_out, device="cuda")
    O3 = torch.zeros(N_out, d, device="cuda")
    L3 = torch.zeros(N_out, device="cuda")

    B_c = 16
    B_r = 16
    T_c = (N_inp + B_c - 1) // B_c
    T_r = (N_out + B_r - 1) // B_r

    int_args = torch.tensor([N_out, T_r, T_c], dtype=torch.int32)
    float_args = torch.tensor([scale_factor], dtype=torch.float32)
    ptr_args = torch.tensor([i.data_ptr() for i in (O3, L3, key, query, value)], dtype=torch.uint64)

    args = torch.tensor([
        *(i.data_ptr() for i in ptr_args),
        *(i.data_ptr() for i in float_args),
        *(i.data_ptr() for i in int_args)], dtype=torch.uint64
    )

    err, _ = cuda.cuLaunchKernel(
        kernel,
        T_r,  # grid x dim
        1,  # grid y dim
        1,  # grid z dim
        32, # block x dim
        32, # block y dim
        1,  # block z dim
        0,  # dynamic shared memory
        torch.cuda.current_stream().stream_id,  # stream
        args.data_ptr(),  # kernel arguments
        0,  # extra (ignore)
    )
    assert err == cuda.CUresult.CUDA_SUCCESS, err
    return O3, L3


/opt/conda/lib/python3.12/site-packages/thunder/executors/transformer_engine_v2ex.py:25: UserWarning: transformer_engine module not found!
  warnings.warn("transformer_engine module not found!")


In [22]:
## Register our implementation as an operator
def my_attn_meta(query, key, value, scale):
    return thunder.TensorProxy(like=query), thunder.TensorProxy(like=query, shape=(query.shape[:-1],))

my_attn = attn_ex.register_operator('my_attn', meta=my_attn_meta, fn=my_attn_impl)

In [23]:
def my_attn_checker(query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False, scale=None):
    if attn_mask is not None or dropout_p == 0.0 or is_causal:
        return False
    if len(query.shape) > 2:
            return (query.device.device_type == thunder.devices.DeviceType.CUDA and
                key.device == query.device and
                value.device == query.device)
    return False

def my_attn_transform(query, key, value, attn_masks=None, dropout_p=0.0, is_causal=False, scale=None):
    if scale is None:
        scale = query.size(-1) ** -0.5
    out = my_attn(query, key, value, scale)
    return out[0]

attn_ex.register_implementation(thunder.torch.scaled_dot_product_attention, checker=my_attn_checker,
                                  execution_transform=my_attn_transform)


#### Run...

In [24]:

def test_fn(query, key, value):
        return torch.nn.functional.scaled_dot_product_attention(query, key, value, is_causal=False)

jfn = thunder.jit(test_fn)

print((jfn(Q, K, V).to("cpu") - test_fn(Q.to("cpu"), K.to("cpu"), V.to("cpu"))).abs().max())
print(thunder.last_traces(jfn)[-1])

tensor(5.6624e-07)
# Constructed by Unwrap the actual return value
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast
def computation(query, key, value):
  # query: "cuda:0 f32[32, 128]"
  # key: "cuda:0 f32[32, 128]"
  # value: "cuda:0 f32[32, 128]"

  # /tmp/ipykernel_17219/2358261693.py:2: 	        return torch.nn.functional.scaled_dot_product_attention(query, key, value, is_causal=False)
  t41 = torch.nn.functional.scaled_dot_product_attention(query, key, value, None, 0.0, False, scale=None)  # t41: "cuda:0 f32[32, 128]"
    # t41 = ltorch.scaled_dot_product_attention(query, key, value, None, 0.0, False, scale=None)  # t41: "cuda:0 f32[32, 128]"
      # t28 = ltorch.mul(query, 0.29730177875068026)  # t28: "cuda:0 f32[32, 128]"
        # t28 = prims.mul(query, 0.29730177875068026)  # t28: "cuda:0 f32[32, 128]"
      # t29 = ltorch.transpose(key, -2, -1)  # t29: "cuda:0 f32[128, 32]"
        # t29 = prims.transpose

## Cuda info

In [25]:
print_cuda_info()

=== PyTorch CUDA Info ===
PyTorch version: 2.6.0
CUDA available: True
CUDA version: 12.6
cuDNN version: 91001
Number of GPUs: 1
  GPU 0: NVIDIA A10G
    Current device: 0
    Memory allocated: 10.97 MB
    Memory cached   : 62.91 MB

=== nvidia-smi Info (if available) ===
Thu Aug 21 12:34:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10G                    On  |   00000